In [6]:
import torch
from torch import nn


class RadioTransformerNetwork(nn.Module):
    def __init__(self, in_channels, compressed_dim):
        super(RadioTransformerNetwork, self).__init__()

        self.in_channels = in_channels

        self.encoder = nn.Sequential(
            nn.Linear(in_channels, in_channels),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels, compressed_dim),
        )

        self.decoder = nn.Sequential(
            nn.Linear(compressed_dim, compressed_dim),
            nn.ReLU(inplace=True),
            nn.Linear(compressed_dim, in_channels)
        )

    def decode_signal(self, x):
        return self.decoder(x)

    def forward(self, x):
        x = self.encoder(x)

        # Normalization.
        x = (self.in_channels ** 2) * (x / x.norm(dim=-1)[:, None])

        # 7dBW to SNR.
        training_signal_noise_ratio = 5.01187

        # bit / channel_use
        communication_rate = 1

        # Simulated Gaussian noise.
        noise = Variable(torch.randn(*x.size()) / ((2 * communication_rate * training_signal_noise_ratio) ** 0.5))
        if USE_CUDA: noise = noise.cuda()
        x += noise

        x = self.decoder(x)

        return x
def get_iterator(mode):
        data = train_data if mode else test_data
        labels = train_labels if mode else test_labels
        tensor_dataset = tnt.dataset.TensorDataset([data, labels])

        return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


def processor(sample):
        data, labels, training = sample

        data = Variable(data)
        labels = Variable(labels)

        if USE_CUDA:
            data = data.cuda()
            labels = labels.cuda()

        outputs = model(data)

        loss = loss_fn(outputs, labels)

        return loss, outputs


def reset_meters():
        meter_accuracy.reset()
        meter_loss.reset()
        confusion_meter.reset()


def on_sample(state):
        state['sample'].append(state['train'])


def on_forward(state):
        meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        meter_loss.add(state['loss'].item())


def on_start_epoch(state):
        reset_meters()
        state['iterator'] = tqdm(state['iterator'])


def on_end_epoch(state):
        print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

        reset_meters()

        engine.test(processor, get_iterator(False))

        print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))
NUM_EPOCHS = 100
BATCH_SIZE = 256
CHANNEL_SIZE = 4
USE_CUDA = False

In [7]:
from tqdm import tqdm
from torchnet.engine import Engine
from torch.autograd import Variable
from torch.optim import Adam
import torchnet as tnt
import math

model = RadioTransformerNetwork(CHANNEL_SIZE, compressed_dim=int(math.log2(CHANNEL_SIZE)))
if USE_CUDA: model = model.cuda()

train_labels = (torch.rand(10000) * CHANNEL_SIZE).long()
train_data = torch.sparse.torch.eye(CHANNEL_SIZE).index_select(dim=0, index=train_labels)

test_labels = (torch.rand(1500) * CHANNEL_SIZE).long()
test_data = torch.sparse.torch.eye(CHANNEL_SIZE).index_select(dim=0, index=test_labels)

optimizer = Adam(model.parameters())

engine = Engine()
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(CHANNEL_SIZE, normalized=True)

loss_fn = nn.CrossEntropyLoss()

In [8]:
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, get_iterator(True), maxepoch=NUM_EPOCHS, optimizer=optimizer)


100%|██████████| 40/40 [00:00<00:00, 207.45it/s]

[Epoch 1] Training Loss: 1.6789 (Accuracy: 24.98%)


[Epoch 1] Testing Loss: 1.3288 (Accuracy: 27.47%)


100%|██████████| 40/40 [00:00<00:00, 229.05it/s]

[Epoch 2] Training Loss: 1.2210 (Accuracy: 24.98%)


...

[Epoch 99] Testing Loss: 0.0328 (Accuracy: 100.00%)


100%|██████████| 40/40 [00:00<00:00, 229.89it/s]

[Epoch 100] Training Loss: 0.0326 (Accuracy: 100.00%)


[Epoch 100] Testing Loss: 0.0318 (Accuracy: 100.00%)


{'network': <function __main__.processor(sample)>,
 'iterator': <tqdm.std.tqdm at 0x7f1ab2c62250>,
 'maxepoch': 100,
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 'epoch': 100,
 't': 4000,
 'train': True,
 'sample': [tensor([[0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 1., 0.],
          [0., 1., 0., 0.],
          [0., 1., 0., 0.],
          [0., 1., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.],
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.]]),
  tensor([2, 0, 2, 2, 0, 2, 2, 1, 1, 1, 0, 2, 3, 3, 1, 2]),
  True],
 'output': None,
 'loss': None}

In [4]:
train_data[:4]

tensor([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]])

In [5]:
train_labels[:10]

tensor([3, 1, 0, 1, 1, 2, 0, 0, 3, 2])